# XOR

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json, matplotlib
#s = json.load( open("styles/bmh_matplotlibrc.json") )
#matplotlib.rcParams.update(s)
from IPython.core.pylabtools import figsize
figsize(11, 5)
colores = ["#348ABD", "#A60628","#06A628"]

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [4]:
import numpy as np

## Función de activación

In [5]:
# Devuelve la función logística evaluada
# componente por componente
def logistica(z):
    return 1 / (1 + np.exp(-z))

In [6]:
## Función que, dado un arreglo de valores z
## calcula el valor de la derivada para cada entrada.

def derivadaLogistica(z):
    g = logistica(z)
    return g * (1 - g)

## Red neuronal
La red implementa encadenamiento hacia adelante (para evaluar) y hacia atrás (para entrenarse).

In [7]:
np.random.seed(10)

In [8]:
class XOR:
    def __init__(self):
        self.Theta_0 = np.random.random((2,3))
        self.Theta_1 = np.random.random((1,3))
    
    def feedForward(self, X, vector = None):
        """ Calcula las salidas, dados los datos de entrada. """
        if vector is None:
            Theta_0 = self.Theta_0
            Theta_1 = self.Theta_1
        else:
            Theta_0, Theta_1 = self.reconstructMatrices(vector)
        
        self.A0 = np.vstack((np.ones((1, X.shape[0])), X.T))
        self.Z1 = np.dot(Theta_0, self.A0)
        self.A1 = np.vstack((np.ones((1, self.Z1.shape[1])), logistica(self.Z1)))
        self.Z2 = np.dot(Theta_1, self.A1)
        self.A2 = logistica(self.Z2)
        
    def backPropagate(self, X, Y):
        """ Calcula el error y su gradiente,
        dados los pesos actuales de la red y los resultados
        esperados.
        """
        self.feedForward(X)
        
        m = X.shape[0]
        Delta_2 = (Y.T - self.A2)
        self.error = np.sum(Delta_2 ** 2) / (2 * m) # Suma renglones (clases) y columnas (ejemplares)
        
        Delta_2 = Delta_2 * derivadaLogistica(self.Z2)
        
        self.Grad_1 = - np.dot(Delta_2, self.A1.T) / m
        
        print("Delta_2")
        print(Delta_2)
        
        print("A1")
        print(self.A1)
        
        print("self.Grad_1")
        print(self.Grad_1)
        
        Delta_1 = np.dot(self.Theta_1[:,1:].T, Delta_2) * derivadaLogistica(self.Z1)
        self.Grad_0 = - np.dot(Delta_1, self.A0.T) / m
        print("Delta_1")
        print(Delta_1)
        
        print("self.Grad_0")
        print(self.Grad_0)
        
    def calcError(self, X, Y, vector):
        """
        Calcula el error que se cometería utilizando los pesos en 'vector'.
        """
        self.feedForward(X, vector)
        m = X.shape[0]
        Delta_2 = (Y.T - self.A2)
        error = np.sum(Delta_2.T ** 2) / (2 * m)
        return error
    
    def vectorWeights(self):
        """
        Acomoda a todos los parámetros en las matrices de pesos, en un solo vector.
        """
        vector = np.vstack((self.Theta_0.reshape((self.Theta_0.size, 1)),
                          self.Theta_1.reshape((self.Theta_1.size, 1))))
        #print(self.Theta_0, self.Theta_1, vector)
        return vector
    
    def reconstructMatrices(self, vector):
        """
        Dado un vector, rearma matrices del tamaño de las matrices de pesos.
        """
        M0 = vector[0:self.Theta_0.size].reshape(self.Theta_0.shape)
        M1 = vector[self.Theta_0.size:].reshape(self.Theta_1.shape)
        return M0, M1
        
    def approxGradient(self, X, Y):
        """
        Aproxima el valor del gradiente alrededor de los pesos actuales,
        perturbando cada valor, uno por uno.
        """
        vector = self.vectorWeights().copy()
        approx = np.zeros(vector.shape)
        perturb = np.zeros(vector.shape)
        epsilon = 0.0001
        
        for i in range(len(vector)):
            perturb[i] = epsilon
            loss1 = self.calcError(X, Y, vector - perturb)
            loss2 = self.calcError(X, Y, vector + perturb)
            perturb[i] = 0
            approx[i] = (loss2 - loss1) / (2 * epsilon)
        return self.reconstructMatrices(approx)
        
    def gradientDescent(self, X, Y, alpha, ciclos=10, checkGradient = False):
        """ Evalúa y ajusta los pesos de la red,
        de acuerdo a los datos en X y los resultados
        esperados, en Y.
        """
        errores = np.zeros(ciclos)
        for i in range(ciclos):
            self.backPropagate(X, Y)
            Grad_1 = self.Grad_1
            Grad_0 = self.Grad_0
            if checkGradient:
                ApproxT0, ApproxT1 = self.approxGradient(X, Y)
                print("Grad 0 = ", Grad_0, "\nApprox = ", ApproxT0, "\nDiff = ", Grad_0 - ApproxT0,
                     "\nGrad 1 = ", Grad_1, "\nApprox = ", ApproxT1, "\nDiff = ", Grad_1 - ApproxT1)
            self.Theta_1 -= alpha * Grad_1
            self.Theta_0 -= alpha * Grad_0
            errores[i] = self.error
        if ciclos > 1:
            plt.plot(np.arange(ciclos), errores)
        
    def printOutput(self):
        print(np.hstack((self.A0.T[:,1:], self.A2.T)))

In [9]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])
xor = XOR()
xor.feedForward(X)
xor.printOutput()

[[0.         0.         0.69697434]
 [0.         1.         0.71737828]
 [1.         0.         0.70117284]
 [1.         1.         0.72086962]]


In [10]:
print(xor.vectorWeights())
print(xor.Theta_0, '\n', xor.Theta_1)
T0, T1 = xor.reconstructMatrices(xor.vectorWeights())
print(T0, T1)

[[0.77132064]
 [0.02075195]
 [0.63364823]
 [0.74880388]
 [0.49850701]
 [0.22479665]
 [0.19806286]
 [0.76053071]
 [0.16911084]]
[[0.77132064 0.02075195 0.63364823]
 [0.74880388 0.49850701 0.22479665]] 
 [[0.19806286 0.76053071 0.16911084]]
[[0.77132064 0.02075195 0.63364823]
 [0.74880388 0.49850701 0.22479665]] [[0.19806286 0.76053071 0.16911084]]


In [11]:
xor.gradientDescent(X, Y, 0.3, 1, checkGradient = True)

Delta_2
[[-0.14720175  0.05730062  0.0626131  -0.14505094]]
A1
[[1.         1.         1.         1.        ]
 [0.68380651 0.80297119 0.68827618 0.8062337 ]
 [0.67891802 0.72583658 0.77683402 0.81337751]]
self.Grad_1
[[0.04308474 0.03212416 0.03192206]]
Delta_1
[[-0.0242056   0.00689455  0.0102168  -0.01723362]
 [-0.00542647  0.00192832  0.00183566 -0.00372347]]
self.Grad_0
[[0.00608197 0.00175421 0.00258477]
 [0.00134649 0.00047195 0.00044879]]
Grad 0 =  [[0.00608197 0.00175421 0.00258477]
 [0.00134649 0.00047195 0.00044879]] 
Approx =  [[0.00608197 0.00175421 0.00258477]
 [0.00134649 0.00047195 0.00044879]] 
Diff =  [[ 3.02381210e-12 -1.00241473e-12  1.61952049e-13]
 [ 3.69688655e-13 -2.09471763e-13 -2.46954475e-13]] 
Grad 1 =  [[0.04308474 0.03212416 0.03192206]] 
Approx =  [[0.04308474 0.03212416 0.03192206]] 
Diff =  [[1.05755252e-10 4.46148743e-11 4.47641507e-11]]


In [12]:
@interact_manual(ciclos = (50, 2000))
def trainXOR(ciclos):
    xor.gradientDescent(X, Y, 0.3, ciclos)

interactive(children=(IntSlider(value=1025, description='ciclos', max=2000, min=50), Button(description='Run I…

In [0]:
xor.feedForward(X)
xor.printOutput()
print("Theta_0 = ", xor.Theta_0, "\nTheta_1", xor.Theta_1)

[[0.         0.         0.04092749]
 [0.         1.         0.96324339]
 [1.         0.         0.96345047]
 [1.         1.         0.03896931]]
Theta_0 =  [[-2.61672855  6.14722615  6.05412519]
 [-6.07131711  3.96839695  3.94959144]] 
Theta_1 [[-3.68916239  8.1602158  -8.8866574 ]]


In [0]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read() #or edit path to custom.css
    return HTML(styles)
css_styling()